# 02a Models Pre Deployment Analysis

Analysis and evaluation of the training data, model size, accuracy with TFL and anything else which can be evaluated *prior* to the deployment on the target hardware.

In [1]:
%run '00_Imports-and-settings.ipynb'

Numpy Version:		 1.19.0
Pandas Version:		 1.0.5
Matplotlib Version:	 3.2.2


In [2]:
# helper function to get model name
def get_model_name(model_string):
    if 'ResNet' in model_string:
        model_name = 'ResNet20_CIFAR-10'
    elif 'LeNet' in model_string:
        model_name = 'LeNet-MNIST'
    else:
        model_name = 'unknown'
        print('unknown model name')
    return model_name

# Static Analysis

## Training Parameters

### **LeNet MNIST**

```python
batch_size = 4000
learning_rate = 0.003
epochs = 50
validation_split = 0.1

 shuffle=True,
(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                 loss="categorical_crossentropy",
                 metrics=['accuracy'])
    
```


### **ResNet20 on CIFAR-10**

```python
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
                               
# Training parameters
BATCH_SIZE = 32  # orig paper trained all networks with batch_size=128
EPOCHS = 200 # 200

USE_AUGMENTATION = True
# Subtracting pixel mean improves accuracy
SUBTRACT_PIXEL_MEAN = False

NUM_CLASSES = np.unique(y_train).shape[0] # 10
COLORS = x_train.shape[3]

# Computed depth from supplied model parameter n
DEPTH = COLORS * 6 + 2                              
                               
```

dynamic learning rate
```python
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
```
data augmentation
```python
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation 
        # (strictly between 0 and 1)
        validation_split=0.0)
```


## Complexity: Parameters, Layers, Computations (MACs), etc.

see `results_energy/analysis_energy_per_layer-complexity.xlsx`

## Accuracy & Loss

### Accuracy Keras

**LeNet5 MNIST**

```python
Test accuracy:	 0.9879000186920166
Test loss:	 0.04126233980059624
```

Loss function: `categorical_crossentropy`


**LeNet5 MNIST pruned**
```python
Test accuracy:		 0.9879000186920166
Test accuracy (pruned):	 0.98089998960495
Test loss:		 0.04126233980059624
Test loss (pruned):	 1.4850536584854126


    
Test crossentropy loss:	 0.062741384    
```

ResNet20 CIFAR-10
```python
Test accuracy:	 0.9185000061988831
Test loss:	 0.4342728555202484
```

Loss function: `categorical_crossentropy`

### Load the data

In [3]:
model = 'LeNet-MNIST'
l4_df = pd.read_pickle('results/L4/NUCLEO_L496ZG_LeNet-MNIST_benchmarking+verification_results_2020-07-24.pkl').sort_values(by=['model_type'])
f4_df = pd.read_pickle('results/F4/DISCO_F469NI_LeNet-MNIST_benchmarking+verification_results_2020-07-18.pkl').sort_values(by=['model_type'])
f7_df = pd.read_pickle('results/F7/NUCLEO_F767ZI_LeNet-MNIST_benchmarking+verification_results_2020-07-22.pkl').sort_values(by=['model_type'])

lenet_df = {"L4": l4_df, "F4": f4_df, "F7": f7_df}


model = '01d_ResNet20_CIFAR-10'
l4_df = pd.read_pickle('results/L4/NUCLEO_L496ZG_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-20.pkl').sort_values(by=['model_type'])
f4_df = pd.read_pickle('results/F4/DISCO_F469NI_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-19.pkl').sort_values(by=['model_type'])
f7_df = pd.read_pickle('results/F7/NUCLEO_F767ZI_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-22.pkl').sort_values(by=['model_type'])

resnet_df = {"L4": l4_df, "F4": f4_df, "F7": f7_df}

Available columns

In [4]:
resnet_df['F4'].columns

Index(['time', 'MCU', 'model', 'mbed-dir', 'cmsis-nn', 'arena_size',
       'compiler_optimization', 'FPU_status', 'MACs', 'pruned', 'weights',
       'activations', 'model_type', 'model_size', 'model_size_reduction',
       'model_size_gzip', 'model_size_reduction_gzip', 'binary_size',
       'input_details_dtype', 'input_details_shape', 'output_details_dtype',
       'output_details_shape', 'tfl_interpreter_accuracy',
       'tfl_interpreter_loss_crossentropy', 'tfl_interpreter_loss_meansquared',
       'inferences_per_cycle', 'tflu_mcu_benchmark_single',
       'tflu_mcu_benchmark_mean', 'tflu_mcu_benchmark_std',
       'tflu_mcu_accuracy', 'tflu_mcu_loss_crossentropy',
       'tflu_mcu_loss_meansquared', 'tflu_mcu_accuracy_50',
       'tflu_mcu_benchmark_mean_50', 'tflu_mcu_benchmark_std_50'],
      dtype='object')

In [5]:
lenet_df['L4'].iloc[0:6]

,time,MCU,model,mbed-dir,cmsis-nn,arena_size,compiler_optimization,FPU_status,MACs,pruned,...,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy,tfl_interpreter_loss_meansquared,inferences_per_cycle,tflu_mcu_benchmark_single,tflu_mcu_benchmark_mean,tflu_mcu_benchmark_std,tflu_mcu_accuracy,tflu_mcu_loss_crossentropy,tflu_mcu_loss_meansquared
0,2020-07-23 22:05:08.702068,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed,none,60,-Ofast,1,161925,0,...,0.9879,0.04126234,0.00190614,10,112249.9,112196.2348,0.42434533,0.9879,0.04126231,0.00190614
1,2020-07-23 22:35:57.563487,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,60,-Ofast,1,161925,0,...,0.9879,0.04126234,0.00190614,10,113205.4,113151.6721,0.46944818,0.9879,0.04126231,0.00190614
4,2020-07-23 23:54:31.932797,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed,none,60,-Ofast,0,161925,0,...,0.9879,0.04719499,0.00193198,10,392183.9,405645.1660,5396.24919749,0.9879,0.04126231,0.00190614
5,2020-07-24 01:14:14.473989,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,60,-Ofast,0,161925,0,...,0.9879,0.04719499,0.00193198,10,391028.6,404489.2701,5396.07043304,0.9879,0.04126231,0.00190614
8,2020-07-24 03:21:15.122575,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed,none,60,-Os,1,161925,0,...,0.9879,0.04719499,0.00193198,10,175906.4,175796.0092,0.12375524,0.9879,0.04126231,0.00190614
9,2020-07-24 04:02:07.557607,NUCLEO_L496ZG,LeNet-MNIST_none_tflite-builtins_none_none,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,60,-Os,1,161925,0,...,0.9879,0.04719499,0.00193198,10,173783.1,173672.6751,0.47427839,0.9879,0.04126231,0.00190614


### Accuracy TFL

In [6]:
for dfs in [lenet_df, resnet_df]:
    for mcu in dfs:
        df = dfs[mcu]
        
        model = df['model'].unique()[0]
        # filter =  (df['weights'] == df['activations'])
        # df_filtered = df.loc[filter]
        df_filtered = df
        print(model)
        display(df[['model_type','activations','tfl_interpreter_accuracy','tfl_interpreter_loss_crossentropy']])

LeNet-MNIST_none_tflite-builtins_none_none


,model_type,activations,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
0,W-float32_A-float32,float32,0.9879,0.04126234
1,W-float32_A-float32,float32,0.9879,0.04126234
4,W-float32_A-float32,float32,0.9879,0.04719499
5,W-float32_A-float32,float32,0.9879,0.04719499
8,W-float32_A-float32,float32,0.9879,0.04719499
9,W-float32_A-float32,float32,0.9879,0.04719499
12,W-float32_A-float32,float32,0.9879,0.04719499
13,W-float32_A-float32,float32,0.9879,0.04719499
2,W-int8_A-int8,int8,0.9874,0.04719499
3,W-int8_A-int8,int8,0.9874,0.04719499


LeNet-MNIST_none_tflite-builtins_none_none


,model_type,activations,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
0,W-float32_A-float32,float32,0.9879,0.04126234
1,W-float32_A-float32,float32,0.9879,0.04126234
4,W-float32_A-float32,float32,0.9879,0.04126234
5,W-float32_A-float32,float32,0.9879,0.04126234
8,W-float32_A-float32,float32,0.9879,0.04126234
9,W-float32_A-float32,float32,0.9879,0.04126234
12,W-float32_A-float32,float32,0.9879,0.04126234
13,W-float32_A-float32,float32,0.9879,0.04126234
2,W-int8_A-int8,int8,0.9874,0.04719499
3,W-int8_A-int8,int8,0.9874,0.04719499


LeNet-MNIST_none_tflite-builtins_none_none


...

,model_type,activations,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
11,W-float32_A-float32,float32,0.9185,0.31073594
17,W-float32_A-float32,float32,0.9185,0.31073594
16,W-float32_A-float32,float32,0.9185,0.31073594
22,W-float32_A-float32,float32,0.9185,0.31073594
10,W-float32_A-float32,float32,0.9185,0.31073594
23,W-float32_A-float32,float32,0.9185,0.31073594
4,W-float32_A-float32,float32,0.9185,0.31073594
5,W-float32_A-float32,float32,0.9185,0.31073594
14,W-int8_A-float32,float32,0.9167,0.31073594
21,W-int8_A-float32,float32,0.9167,0.31073594


01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none


,model_type,activations,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
11,W-float32_A-float32,float32,0.9185,0.31073594
17,W-float32_A-float32,float32,0.9185,0.31073594
16,W-float32_A-float32,float32,0.9185,0.31073594
22,W-float32_A-float32,float32,0.9185,0.31073594
10,W-float32_A-float32,float32,0.9185,0.31073594
23,W-float32_A-float32,float32,0.9185,0.31073594
4,W-float32_A-float32,float32,0.9185,0.31073594
5,W-float32_A-float32,float32,0.9185,0.31073594
14,W-int8_A-float32,float32,0.9167,0.31073594
21,W-int8_A-float32,float32,0.9167,0.31073594


01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none


,model_type,activations,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
2,W-float32_A-float32,float32,0.9185,0.31073594
3,W-float32_A-float32,float32,0.9185,0.31073594
6,W-float32_A-float32,float32,0.9185,0.31073594
7,W-float32_A-float32,float32,0.9185,0.31073594
10,W-float32_A-float32,float32,0.9185,0.31073594
11,W-float32_A-float32,float32,0.9185,0.31073594
14,W-float32_A-float32,float32,0.9185,0.31073594
15,W-float32_A-float32,float32,0.9185,0.31073594
0,W-int8_A-int8,int8,0.9179,0.40206301
1,W-int8_A-int8,int8,0.9179,0.40206301


Load one older model for the results for mixed quantization for LeNet

In [7]:

df = pd.read_pickle('results/L4/preliminaries/NUCLEO_L496ZG_LeNet-MNIST_benchmarking+verification_+pruned_results_2020-07-01.pkl')

df_filtered = df
display(df[['pruned','model_type','tfl_interpreter_accuracy','tfl_interpreter_loss_crossentropy']].drop_duplicates())

,pruned,model_type,tfl_interpreter_accuracy,tfl_interpreter_loss_crossentropy
0,1,W-int8_A-float32,0.9855,0.05463612
2,0,W-int8_A-float32,0.9877,0.04129561
4,0,W-int8_A-int8,0.9874,0.04719499
6,0,W-float32_A-float32,0.9879,0.04126234
8,1,W-int8_A-int8,0.9855,0.06460178
10,1,W-float32_A-float32,0.9856,0.05447171


## Results from the Benchmarking on MCU

### Accuracy TFLu

In [8]:
for dfs in [lenet_df, resnet_df]:
    for mcu in dfs:
        df = dfs[mcu]
        
        model = df['model'].unique()[0]
        # filter =  (df['weights'] == df['activations'])
        # df_filtered = df.loc[filter]
        df_filtered = df
        print(model)
        display(df[['model_type','cmsis-nn','FPU_status','tfl_interpreter_accuracy','tflu_mcu_accuracy', 'tfl_interpreter_loss_crossentropy', 'tflu_mcu_loss_crossentropy']].drop_duplicates()) # 'tflu_mcu_accuracy_50'

LeNet-MNIST_none_tflite-builtins_none_none


,model_type,cmsis-nn,FPU_status,tfl_interpreter_accuracy,tflu_mcu_accuracy,tfl_interpreter_loss_crossentropy,tflu_mcu_loss_crossentropy
0,W-float32_A-float32,none,1,0.9879,0.9879,0.04126234,0.04126231
1,W-float32_A-float32,cmsis-nn,1,0.9879,0.9879,0.04126234,0.04126231
4,W-float32_A-float32,none,0,0.9879,0.9879,0.04719499,0.04126231
5,W-float32_A-float32,cmsis-nn,0,0.9879,0.9879,0.04719499,0.04126231
8,W-float32_A-float32,none,1,0.9879,0.9879,0.04719499,0.04126231
9,W-float32_A-float32,cmsis-nn,1,0.9879,0.9879,0.04719499,0.04126231
2,W-int8_A-int8,none,1,0.9874,0.9874,0.04719499,0.04719499
3,W-int8_A-int8,cmsis-nn,1,0.9874,0.9876,0.04719499,0.04717492
6,W-int8_A-int8,none,0,0.9874,0.9874,0.04719499,0.04719499
7,W-int8_A-int8,cmsis-nn,0,0.9874,0.9876,0.04719499,0.04717492


LeNet-MNIST_none_tflite-builtins_none_none


,model_type,cmsis-nn,FPU_status,tfl_interpreter_accuracy,tflu_mcu_accuracy,tfl_interpreter_loss_crossentropy,tflu_mcu_loss_crossentropy
0,W-float32_A-float32,none,0,0.9879,0.9879,0.04126234,0.04126231
1,W-float32_A-float32,cmsis-nn,0,0.9879,0.9879,0.04126234,0.04126231
4,W-float32_A-float32,none,1,0.9879,0.9879,0.04126234,0.04126231
5,W-float32_A-float32,cmsis-nn,1,0.9879,0.9879,0.04126234,0.04126231
2,W-int8_A-int8,none,0,0.9874,0.9874,0.04719499,0.04719499
3,W-int8_A-int8,cmsis-nn,0,0.9874,0.9876,0.04719499,0.04717492
6,W-int8_A-int8,none,1,0.9874,0.9874,0.04719499,0.04719499
7,W-int8_A-int8,cmsis-nn,1,0.9874,0.9876,0.04719499,0.04717492


LeNet-MNIST_none_tflite-builtins_none_none


...

,model_type,cmsis-nn,FPU_status,tfl_interpreter_accuracy,tflu_mcu_accuracy,tfl_interpreter_loss_crossentropy,tflu_mcu_loss_crossentropy
11,W-float32_A-float32,cmsis-nn,0,0.9185,NaN,0.31073594,NaN
17,W-float32_A-float32,cmsis-nn,1,0.9185,NaN,0.31073594,NaN
16,W-float32_A-float32,none,1,0.9185,NaN,0.31073594,NaN
22,W-float32_A-float32,none,0,0.9185,NaN,0.31073594,NaN
14,W-int8_A-float32,none,1,0.9167,NaN,0.31073594,NaN
21,W-int8_A-float32,cmsis-nn,0,0.9167,NaN,0.31073594,NaN
8,W-int8_A-float32,none,0,0.9167,NaN,0.31073594,NaN
3,W-int8_A-float32,cmsis-nn,1,0.9167,NaN,0.31606001,NaN
15,W-int8_A-float32,cmsis-nn,1,0.9167,NaN,0.31073594,NaN
2,W-int8_A-float32,none,1,0.9167,NaN,0.31606001,NaN


01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none


,model_type,cmsis-nn,FPU_status,tfl_interpreter_accuracy,tflu_mcu_accuracy,tfl_interpreter_loss_crossentropy,tflu_mcu_loss_crossentropy
11,W-float32_A-float32,cmsis-nn,0,0.9185,NaN,0.31073594,NaN
17,W-float32_A-float32,cmsis-nn,1,0.9185,NaN,0.31073594,NaN
16,W-float32_A-float32,none,1,0.9185,NaN,0.31073594,NaN
22,W-float32_A-float32,none,0,0.9185,NaN,0.31073594,NaN
14,W-int8_A-float32,none,1,0.9167,NaN,0.31073594,NaN
21,W-int8_A-float32,cmsis-nn,0,0.9167,NaN,0.31073594,NaN
8,W-int8_A-float32,none,0,0.9167,NaN,0.31073594,NaN
3,W-int8_A-float32,cmsis-nn,1,0.9167,NaN,0.31606001,NaN
15,W-int8_A-float32,cmsis-nn,1,0.9167,NaN,0.31073594,NaN
2,W-int8_A-float32,none,1,0.9167,NaN,0.31606001,NaN


01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none


,model_type,cmsis-nn,FPU_status,tfl_interpreter_accuracy,tflu_mcu_accuracy,tfl_interpreter_loss_crossentropy,tflu_mcu_loss_crossentropy
2,W-float32_A-float32,none,1,0.9185,NaN,0.31073594,NaN
3,W-float32_A-float32,cmsis-nn,1,0.9185,NaN,0.31073594,NaN
6,W-float32_A-float32,none,0,0.9185,NaN,0.31073594,NaN
7,W-float32_A-float32,cmsis-nn,0,0.9185,NaN,0.31073594,NaN
0,W-int8_A-int8,none,1,0.9179,NaN,0.40206301,NaN
1,W-int8_A-int8,cmsis-nn,1,0.9179,NaN,0.40206301,NaN
4,W-int8_A-int8,none,0,0.9179,NaN,0.31073594,NaN
5,W-int8_A-int8,cmsis-nn,0,0.9179,NaN,0.31073594,NaN
8,W-int8_A-int8,none,1,0.9179,NaN,0.31073594,NaN
9,W-int8_A-int8,cmsis-nn,1,0.9179,NaN,0.31073594,NaN


## Size

#### Quantization

In [9]:
# overwrite lenet_df to also get mixed model results
lenet_df_with_mixed = {}
lenet_df_with_mixed['L4'] = pd.read_pickle('results/L4/preliminaries/NUCLEO_L496ZG_LeNet-MNIST_benchmarking+verification_+pruned_results_2020-07-01.pkl')


for dfs in [lenet_df_with_mixed, resnet_df]:
        # we dont' need to loop over the mcus as the size is independent of the target
        df = dfs['L4']

        model_name = get_model_name(df['model'].unique()[0])
        
        # filter =  (df['weights'] == df['activations'])
        # df_filtered = df.loc[filter]
        df_filtered = df
        
        # let's sort the filtered df so that the order in the plots match
        df_filtered = df_filtered.sort_values(by=['model_type'])

        fig = sns.catplot(y='model_type', x='model_size', kind='bar', data=df_filtered, height=4, aspect=16/9)
        

        
#         plt.title(model_name)

        plt.xlabel("Size [KiB]")
        plt.ylabel("Model Type")
        fig.set_yticklabels(['\\texttt{float32}', '\\texttt{float32}; \\texttt{int8}', '\\texttt{int8}'])
        filename = f'figures/size/{model_name}_size_quantization'
        
        print(filename)
        display(df_filtered[['model_type','model_size']].drop_duplicates())

        fig = fig.fig
        plt.tight_layout()
        fig.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/size/LeNet-MNIST_size_quantization


,model_type,model_size
23,W-float32_A-float32,320.265625
12,W-int8_A-float32,88.531250
45,W-int8_A-int8,85.187500


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/size/ResNet20_CIFAR-10_size_quantization


,model_type,model_size
11,W-float32_A-float32,1085.17578125
2,W-int8_A-float32,297.56250000
19,W-int8_A-int8,320.29687500


#### Pruning

In [10]:
# load a previous dataset where the pruning model was still benchmarked
lenet_df_with_pruning = pd.read_pickle('results/L4/preliminaries/NUCLEO_L496ZG_LeNet-MNIST_benchmarking+verification_+pruned_results_2020-07-01.pkl')
df = lenet_df_with_pruning

model = df['model'].unique()[0]

#filter =  (df['pruned'] == 1)
#df_filtered = df.loc[filter]
df_filtered = df
df_filtered = df_filtered.sort_values(by=['model_type'])

di = {0: "original", 1: "pruned"}
df_filtered = df_filtered.replace({"pruned": di})

fig = sns.catplot(y='model_type', x='model_size_gzip', kind='bar',hue='pruned', data=df_filtered, height=4, aspect=16/9, legend_out=False)

sns.despine()

plt.xlabel("Model Size gzipped [KiB]")
plt.ylabel("Model Type")
fig.set_yticklabels(['\\texttt{float32}', '\\texttt{float32}; \\texttt{int8}', '\\texttt{int8}'])


filename = f'figures/size/L4_LeNet_size_pruning'

print(filename)
display(df_filtered[['model_type','model_size_gzip']].drop_duplicates())

fig = fig.fig
#plt.tight_layout()
fig.savefig(filename + '.pdf')
# tikzplotlib.clean_figure()
tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/size/L4_LeNet_size_pruning


,model_type,model_size_gzip
23,W-float32_A-float32,92.05175781
43,W-float32_A-float32,295.92675781
12,W-int8_A-float32,30.09863281
2,W-int8_A-float32,70.83593750
45,W-int8_A-int8,28.77246094
41,W-int8_A-int8,65.80957031


#### Binary Size
*todo* could we add the size of the model to the plots?

In [11]:


for dfs in [lenet_df, resnet_df]:
    for mcu in dfs:
        df = dfs[mcu]
        model_name = get_model_name(df['model'].unique()[0])

        filter =  (df['weights'] == df['activations'])
        df_filtered = df.loc[filter]
        #df_filtered = df.copy()
        df_filtered.loc[:,('binary_size')] /= 1024
        
        fig, ax = plt.subplots(figsize=(5,4))

        sns.barplot(ax=ax, y='model_type', x='binary_size',hue='compiler_optimization', data=df_filtered)#, label="Binary Size")#, height=4, aspect=16/9)
        
        sns.barplot(ax=ax, y='model_type', x='model_size', data=df_filtered,  color="g", label="Model Size")#, height=4, aspect=16/9)
        display(df_filtered[['model_type', 'binary_size']])

#         ax.set_title(model_name)
        ax.set_xlabel("Binary Size [KiB]")
        ax.set_ylabel("Model Type")
        ax.set_yticklabels(['\\texttt{float32}', '\\texttt{int8}'])


        ax.legend()
        filename = f'figures/size/{mcu}_{model_name}_size_binary'
        sns.despine()
        plt.tight_layout()
        fig.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,binary_size
0,W-float32_A-float32,593.04688
1,W-float32_A-float32,611.64062
4,W-float32_A-float32,595.79688
5,W-float32_A-float32,614.32812
8,W-float32_A-float32,474.1875
9,W-float32_A-float32,490.52344
12,W-float32_A-float32,474.6875
13,W-float32_A-float32,490.96094
2,W-int8_A-int8,357.97656
3,W-int8_A-int8,376.57031


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,binary_size
0,W-float32_A-float32,593.0625
1,W-float32_A-float32,611.65625
4,W-float32_A-float32,593.0625
5,W-float32_A-float32,611.65625
8,W-float32_A-float32,593.0625
9,W-float32_A-float32,611.65625
12,W-float32_A-float32,593.0625
13,W-float32_A-float32,611.65625
2,W-int8_A-int8,593.0625
3,W-int8_A-int8,611.65625


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

,model_type,binary_size
11,W-float32_A-float32,NaN
17,W-float32_A-float32,NaN
16,W-float32_A-float32,NaN
22,W-float32_A-float32,NaN
10,W-float32_A-float32,NaN
23,W-float32_A-float32,NaN
4,W-float32_A-float32,NaN
5,W-float32_A-float32,NaN
19,W-int8_A-int8,491.00781
18,W-int8_A-int8,474.73438


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,binary_size
11,W-float32_A-float32,1374.9922
17,W-float32_A-float32,1250.8672
16,W-float32_A-float32,1234.5391
22,W-float32_A-float32,1235.0391
10,W-float32_A-float32,1356.3984
23,W-float32_A-float32,1251.3047
4,W-float32_A-float32,1353.6484
5,W-float32_A-float32,1372.2422
19,W-int8_A-int8,486.4375
18,W-int8_A-int8,470.16406


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,binary_size
2,W-float32_A-float32,1345.5312
3,W-float32_A-float32,1364.75
6,W-float32_A-float32,1355.9688
7,W-float32_A-float32,1375.125
10,W-float32_A-float32,1231.3047
11,W-float32_A-float32,1247.7656
14,W-float32_A-float32,1239.0391
15,W-float32_A-float32,1255.4375
0,W-int8_A-int8,580.66406
1,W-int8_A-int8,599.88281


# Memory Usage while running
**TFLite Model Analysis tool**

based on https://github.com/eliberis/tflite-tools

## LeNet

In [13]:
!python ~/masterthesis/tools/tflite-tools/tflite_tools.py -i ~/masterthesis/jupyter-workspace/LeNet5-on-MNIST/TFLite-model/LeNet-MNIST_none_tflite-builtins_none_none.tflite --plot test.png

Tensor information (weights excluded):
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
| Id |                                      Tensor                                      |      Shape      | Size in RAM (B) |
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
|  0 |                                  conv2d_4_input                                  |  (1, 32, 32, 1) |           4,096 |
| 12 | sequential_2/conv2d_4/Relu;sequential_...ntial_2/conv2d_4/BiasAdd/ReadVariableOp |  (1, 30, 30, 6) |          21,600 |
| 13 |                     sequential_2/average_pooling2d_4/AvgPool                     |  (1, 15, 15, 6) |           5,400 |
| 14 | sequential_2/conv2d_5/Relu;sequential_...ntial_2/conv2d_5/BiasAdd/ReadVariableOp | (1, 13, 13, 16) |          10,816 |
| 15 |                     sequential_2/average_pooling2d_5/AvgPool            

In [14]:
!python ~/masterthesis/tools/tflite-tools/tflite_tools.py -i /Users/nope/masterthesis/jupyter-workspace/LeNet5-on-MNIST/TFLite-model/LeNet-MNIST_none_tflite-builtins-INT8_none_dataset.tflite

Tensor information (weights excluded):
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
| Id |                                      Tensor                                      |      Shape      | Size in RAM (B) |
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
|  0 |                               conv2d_4_input_int8                                |  (1, 32, 32, 1) |           1,024 |
| 12 | sequential_2/conv2d_4/Relu;sequential_...ntial_2/conv2d_4/BiasAdd/ReadVariableOp |  (1, 30, 30, 6) |           5,400 |
| 13 |                     sequential_2/average_pooling2d_4/AvgPool                     |  (1, 15, 15, 6) |           1,350 |
| 14 | sequential_2/conv2d_5/Relu;sequential_...ntial_2/conv2d_5/BiasAdd/ReadVariableOp | (1, 13, 13, 16) |           2,704 |
| 15 |                     sequential_2/average_pooling2d_5/AvgPool            

## ResNet20

In [15]:
!python ~/masterthesis/tools/tflite-tools/tflite_tools.py -i /Users/nope/masterthesis/jupyter-workspace/LeNet5-on-MNIST/TFLite-model/01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none.tflite

Tensor information (weights excluded):
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
| Id |                                      Tensor                                      |      Shape      | Size in RAM (B) |
+----+----------------------------------------------------------------------------------+-----------------+-----------------+
|  0 |                                     input_1                                      |  (1, 32, 32, 3) |          12,288 |
| 46 | model/activation/Relu;model/batch_norm...D/ReadVariableOp;model/conv2d_1/Conv2D1 | (1, 32, 32, 16) |          65,536 |
| 47 | model/activation_1/Relu;model/batch_no...D/ReadVariableOp;model/conv2d_2/Conv2D1 | (1, 32, 32, 16) |          65,536 |
| 48 | model/batch_normalization_2/FusedBatch...D/ReadVariableOp;model/conv2d_3/Conv2D1 | (1, 32, 32, 16) |          65,536 |
| 49 |                      model/activation_2/Relu;model/add/add              

---